In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC as svm
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier as kn
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
from sklearn import tree
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dsG = pd.read_csv('/content/drive/MyDrive/datasets/Project5/gender_submission.csv')
dsTr = pd.read_csv('/content/drive/MyDrive/datasets/Project5/train (1).csv')
dsTe = pd.read_csv('/content/drive/MyDrive/datasets/Project5/test.csv')

#Survivel

In [5]:
dsG.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [6]:
dsG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB


In [7]:
dsG.shape

(418, 2)

In [8]:
dsG.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.363636
std,120.810458,0.481622
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


In [9]:
dsG.isnull().sum()

,0
PassengerId,0
Survived,0


#Test

In [10]:
dsTe.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [11]:
dsTe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [12]:
dsTe.shape

(418, 11)

In [13]:
dsTe.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [14]:
dsTe.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [15]:
dsTe = dsTe.drop(['Cabin'], axis=1)

In [16]:
dsTe.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Embarked,0


In [17]:
dsTe.shape

(418, 10)

In [18]:
dsTr.shape

(891, 12)

In [19]:
dsTr.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [20]:
dsTr = dsTr.drop(['Cabin'], axis=1)

In [21]:
dsTr.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [22]:
dsTe.shape

(418, 10)

#searching for noise in data

In [23]:
dsTe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 32.8+ KB


In [24]:
dsTe = dsTe.drop(['Name', 'Ticket'], axis=1)
dsTr = dsTr.drop(['Name', 'Ticket'], axis=1)

In [25]:
dsTe.shape

(418, 8)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [27]:
dsTe.shape

(418, 8)

In [28]:
df_Tr = pd.get_dummies(dsTr, columns=['Sex'], drop_first=True)

In [29]:
df_Te = pd.get_dummies(dsTe, columns=['Sex'], drop_first=True)

In [30]:
df_Te.shape

(418, 8)

In [31]:
df_Tr.head(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Sex_male
0,1,0,3,22.0,1,0,7.2500,S,True
1,2,1,1,38.0,1,0,71.2833,C,False
2,3,1,3,26.0,0,0,7.9250,S,False
3,4,1,1,35.0,1,0,53.1000,S,False
4,5,0,3,35.0,0,0,8.0500,S,True


In [32]:
df_Tr.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2
Sex_male,0


In [33]:
df_Te.shape

(418, 8)

In [34]:
print(df_Te['Embarked'])

0      Q
1      S
2      Q
3      S
4      S
      ..
413    S
414    C
415    S
416    S
417    C
Name: Embarked, Length: 418, dtype: object


In [35]:
print(df_Tr['Embarked'])

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object


In [36]:
df_Tr2 = df_Tr['Embarked'].fillna(df_Tr['Embarked'].mode()[0], inplace=True)
df_Te2 = df_Te['Embarked'].fillna(df_Te['Embarked'].mode()[0], inplace=True)

df_Tr2 = df_Tr['Age'].fillna(df_Tr['Age'].mean(), inplace=True)
df_Te2 = df_Te['Age'].fillna(df_Te['Age'].mean(), inplace=True)
df_Te2 = df_Te['Fare'].fillna(df_Te['Fare'].mean(), inplace=True)

# Remove inplace=True and reassign the result back to the DataFrames
df_Tr['Embarked'] = df_Tr['Embarked'].fillna(df_Tr['Embarked'].mode()[0])
df_Te['Embarked'] = df_Te['Embarked'].fillna(df_Te['Embarked'].mode()[0])

# Now use df_Tr and df_Te for creating dummies
df_Tr2 = pd.get_dummies(df_Tr, columns=['Embarked'], drop_first=True)
df_Te2 = pd.get_dummies(df_Te, columns=['Embarked'], drop_first=True)

<ipython-input-36-7c7d4aaaa487>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Tr2 = df_Tr['Embarked'].fillna(df_Tr['Embarked'].mode()[0], inplace=True)
<ipython-input-36-7c7d4aaaa487>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [37]:
df_Tr2.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Age,0
SibSp,0
Parch,0
Fare,0
Sex_male,0
Embarked_Q,0
Embarked_S,0


In [38]:
df_Te2.shape

(418, 9)

In [39]:
dsG.shape

(418, 2)

In [40]:
y_test = dsG['Survived']

In [41]:
df_Tr1 = df_Tr2
df_Te1 = df_Te2

In [42]:
df_Te1.isnull().sum()

,0
PassengerId,0
Pclass,0
Age,0
SibSp,0
Parch,0
Fare,0
Sex_male,0
Embarked_Q,0
Embarked_S,0


In [43]:
df_Tr1.iloc[:, -3:] = df_Tr1.iloc[:, -3:].astype(int)
df_Te1.iloc[:, -3:] = df_Te1.iloc[:, -3:].astype(int)

<ipython-input-43-5bf8e1fa22d8>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex_male, Length: 891, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_Tr1.iloc[:, -3:] = df_Tr1.iloc[:, -3:].astype(int)
<ipython-input-43-5bf8e1fa22d8>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      0
1      0
2      0
3      0
4      0
      ..
886    0
887    0
888    0
889    0
890    1
Name: Embarked_Q, Length: 891, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_Tr1.iloc[:, -3:] = df_Tr1.iloc[:, -3:].astype(int)
<ipython-input-43-5bf8e1fa22d8>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a fu

In [44]:
df_Tr1.head(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,1,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,1


In [45]:
df_Tr1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          891 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Sex_male     891 non-null    int64  
 8   Embarked_Q   891 non-null    int64  
 9   Embarked_S   891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.7 KB


In [46]:
df_Te1.head(5)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,1,0
1,893,3,47.0,1,0,7.0000,0,0,1
2,894,2,62.0,0,0,9.6875,1,1,0
3,895,3,27.0,0,0,8.6625,1,0,1
4,896,3,22.0,1,1,12.2875,0,0,1


In [47]:
x_train = df_Tr1.drop(columns=['Survived'])
y_train = df_Tr1['Survived']

In [48]:
 x_test = df_Te1

In [49]:
x_test.isnull().sum()

,0
PassengerId,0
Pclass,0
Age,0
SibSp,0
Parch,0
Fare,0
Sex_male,0
Embarked_Q,0
Embarked_S,0


In [50]:
model = RF(n_estimators=20, max_depth=100,random_state=64)
model.fit(x_train, y_train)
pred = model.predict(x_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)
print(classification_report(pred, y_test, labels=None, digits=4))

0.7894736842105263
              precision    recall  f1-score   support

           0     0.8647    0.8156    0.8394       282
           1     0.6579    0.7353    0.6944       136

    accuracy                         0.7895       418
   macro avg     0.7613    0.7754    0.7669       418
weighted avg     0.7974    0.7895    0.7922       418



In [51]:
model = GaussianNB()
model.fit(x_train, y_train)
pred = model.predict(x_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)
print(classification_report(y_test, pred))

0.9114832535885168
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       266
           1       0.84      0.93      0.88       152

    accuracy                           0.91       418
   macro avg       0.90      0.92      0.91       418
weighted avg       0.92      0.91      0.91       418



In [52]:
model = XGBClassifier()
model=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=1, n_estimators=150, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
model.fit(x_train, y_train)
pred = model.predict(x_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)
print(classification_report(y_test, pred))

0.8301435406698564
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       266
           1       0.80      0.70      0.75       152

    accuracy                           0.83       418
   macro avg       0.82      0.80      0.81       418
weighted avg       0.83      0.83      0.83       418



In [53]:
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)

pred = model.predict(x_test)

accuracy = accuracy_score(pred, y_test)
print(accuracy)
print(classification_report(pred, y_test, labels=None, digits=4))

0.7583732057416268
              precision    recall  f1-score   support

           0     0.8609    0.7816    0.8193       293
           1     0.5789    0.7040    0.6354       125

    accuracy                         0.7584       418
   macro avg     0.7199    0.7428    0.7273       418
weighted avg     0.7766    0.7584    0.7643       418



In [54]:
model = RF(criterion = "gini", max_depth = 100, min_samples_split=9, random_state=54)
model.fit(x_train , y_train)
pred = model.predict(x_test)

accuracy = accuracy_score(y_test,pred)
print(accuracy)
print(classification_report(y_test,pred, labels=None, digits=4))

0.7703349282296651
              precision    recall  f1-score   support

           0     0.7993    0.8534    0.8255       266
           1     0.7090    0.6250    0.6643       152

    accuracy                         0.7703       418
   macro avg     0.7541    0.7392    0.7449       418
weighted avg     0.7664    0.7703    0.7669       418



In [55]:
model = svm()
model.fit(x_train, y_train)
pred = model.predict(x_test)
accuracy = accuracy_score(y_test,pred)
print(accuracy)
print(classification_report(y_test,pred, labels=None, digits=4))

0.6483253588516746
              precision    recall  f1-score   support

           0     0.6484    0.9774    0.7796       266
           1     0.6471    0.0724    0.1302       152

    accuracy                         0.6483       418
   macro avg     0.6477    0.5249    0.4549       418
weighted avg     0.6479    0.6483    0.5435       418



In [56]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
%time model.fit(x_train, y_train)
pred = model.predict(x_test)

accuracy = accuracy_score(y_test,pred)
print(accuracy)
print(classification_report(y_test,pred, labels=None, digits=4))

CPU times: user 6.57 ms, sys: 53 µs, total: 6.63 ms
Wall time: 8.55 ms
0.6578947368421053
              precision    recall  f1-score   support

           0     0.6640    0.9361    0.7769       266
           1     0.6047    0.1711    0.2667       152

    accuracy                         0.6579       418
   macro avg     0.6343    0.5536    0.5218       418
weighted avg     0.6424    0.6579    0.5914       418

